In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score, mean_squared_error, accuracy_score
import xgboost as xgb
import time
import random as rnd

standarscaler = StandardScaler()

import warnings
warnings.filterwarnings("ignore")

In [2]:
TrainData = pd.read_csv('count_train.csv')
X = TrainData.drop('yC', axis = 1)
Y = TrainData['yC']

Xs = standarscaler.fit_transform(X)
Xs = pd.DataFrame(Xs, index=X.index, columns=X.columns.values)

In [3]:
TestData = pd.read_csv('count_test.csv')
Xt = TestData.drop('yC', axis = 1)
Yt = TestData['yC']

Xts = standarscaler.transform(Xt)
Xts = pd.DataFrame(Xts, index=Xt.index, columns=Xt.columns.values)

In [4]:
variables = []
for i in range(1000000):
    variables.append(rnd.sample(X.columns.values.tolist(), rnd.sample([x+1 for x in range(X.shape[1])],1)[0]))

In [5]:
RMSE_List = []
RMSE_List_round = []
RMSE_List_floor = []
RMSE_List_ceil = []
start_time = time.time()
for i in range(len(variables)):
    xg_reg = xgb.XGBRegressor(objective ='reg:squarederror')
    xg_reg.fit(Xs[variables[i]],Y)
    preds = xg_reg.predict(Xts[variables[i]])
    RMSE_List.append(np.sqrt(mean_squared_error(Yt, preds)))
    RMSE_List_round.append(np.sqrt(mean_squared_error(Yt, np.round(preds))))
    RMSE_List_floor.append(np.sqrt(mean_squared_error(Yt, np.floor(preds))))
    RMSE_List_ceil.append(np.sqrt(mean_squared_error(Yt, np.ceil(preds))))
elapsed_time = time.time() - start_time
print(elapsed_time)

38837.91699504852


In [6]:
print(min(RMSE_List))
print(min(RMSE_List_round))
print(min(RMSE_List_floor))
print(min(RMSE_List_ceil))

1.054253958111228
1.0327955589886444
1.1690451944500122
1.1105554165971787


In [47]:
chosen = np.array(variables)[RMSE_List_round == min(RMSE_List_round)]
chosen = chosen.tolist()[0]
chosen

['x4', 'x13', 'x25', 'x24', 'x27', 'x7', 'x14', 'x3', 'x2', 'x8', 'x22']

In [46]:
xg_reg2 = xgb.XGBRegressor(objective ='reg:squarederror')
xg_reg2.fit(Xs[chosen],Y)
preds2 = xg_reg2.predict(Xts[chosen])
np.sqrt(mean_squared_error(Yt, np.round(preds2)))

1.0327955589886444

In [50]:
np.round(preds2)

array([1., 2., 1., 1., 1., 1., 1., 1., 1., 1., 2., 0., 1., 0., 1., 2., 1.,
       1., 3., 2., 2., 1., 1., 0., 1., 1., 4., 1., 1., 1.], dtype=float32)

In [52]:
Yt.values

array([0, 2, 2, 2, 0, 1, 0, 0, 1, 0, 4, 0, 1, 1, 0, 1, 2, 2, 3, 2, 3, 0,
       0, 0, 0, 2, 7, 0, 0, 1], dtype=int64)